In [36]:
# Loading the pretrained model and processor for Information Extraction task
ie_model_ckpt = "naver-clova-ix/donut-base-finetuned-cord-v2"

ie_model = VisionEncoderDecoderModel.from_pretrained(ie_model_ckpt)
ie_processor = DonutProcessor.from_pretrained(ie_model_ckpt)  # restart the session if it is causing any error

config.json:   0%|          | 0.00/4.74k [00:00<?, ?B/s]

C:\Users\rushi\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rushi\.cache\huggingface\hub\models--naver-clova-ix--donut-base-finetuned-cord-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/806M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/806M [00:00<?, ?B/s]

Config of the encoder: <class 'transformers.models.donut.modeling_donut_swin.DonutSwinModel'> is overwritten by shared encoder config: DonutSwinConfig {
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    14,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 1024,
  "image_size": [
    1280,
    960
  ],
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "donut-swin",
  "num_channels": 3,
  "num_heads": [
    4,
    8,
    16,
    32
  ],
  "num_layers": 4,
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "transformers_version": "4.48.0",
  "use_absolute_embeddings": false,
  "window_size": 10
}

Config of the decoder: <class 'transformers.models.mbart.modeling_mbart.MBartForCausalLM'> is overwritten by shared decoder config: MBartConfig {
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_cross_attention": true,
  "add_

...

tokenizer_config.json:   0%|          | 0.00/536 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.02M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

In [37]:
ie_dataset = load_dataset("hf-internal-testing/example-documents", split='test')
ie_dataset

dataset_infos.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

C:\Users\rushi\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rushi\.cache\huggingface\hub\datasets--hf-internal-testing--example-documents. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


(…)-00000-of-00001-cca0db7e388ce4bc.parquet:   0%|          | 0.00/2.86M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/3 [00:00<?, ? examples/s]

Dataset({
    features: ['image'],
    num_rows: 3
})

In [56]:
from datasets import Dataset, Image

# List all image file paths
image_paths = ["Internship_Indoco.jpg"]

# Create a dataset from the image paths
dataset = Dataset.from_dict({"image": image_paths}).cast_column("image", Image())

# Access an image
ie_image = dataset[0]['image']
ie_image.show()


In [57]:
# Donut processor require RGB image, so convert it before feeding.
ie_pixel_values = ie_processor(ie_image.convert("RGB"), return_tensors="pt").pixel_values
ie_pixel_values.size()

torch.Size([1, 3, 1280, 960])

In [58]:
# Prepare decoder inputs
ie_task_prompt = "<s_cord-v2>"
ie_decoder_input_ids = ie_processor.tokenizer(ie_task_prompt, add_special_tokens=False, return_tensors="pt").input_ids
ie_decoder_input_ids

tensor([[57579]])

In [59]:
ie_processor.tokenizer.convert_ids_to_tokens(ie_decoder_input_ids[0])

['<s_cord-v2>']

In [60]:
# Information Extraction using pretrianed model
ie_outputs = ie_model.generate(
    ie_pixel_values.to(device),
    decoder_input_ids=ie_decoder_input_ids.to(device),
    max_length=ie_model.decoder.config.max_position_embeddings,
    pad_token_id=ie_processor.tokenizer.pad_token_id,
    eos_token_id=ie_processor.tokenizer.eos_token_id,
    use_cache=True,
    bad_words_ids=[[ie_processor.tokenizer.unk_token_id]],
    return_dict_in_generate=True,
)

ie_sequence = ie_processor.batch_decode(ie_outputs.sequences)[0]
ie_sequence = ie_sequence.replace(ie_processor.tokenizer.eos_token, "").replace(ie_processor.tokenizer.pad_token, "")
ie_sequence = re.sub(r"<.*?>", "", ie_sequence, count=1).strip()  # remove first task start token
print(ie_processor.token2json(ie_sequence))

{'menu': [{'nm': 'INDOCO REMEDIES LIMITED', 'unitprice': '196', 'cnt': {'unitprice': '196'}, 'price': '(91-240) 255 6297', 'sub': {'nm': 'B-20. M.I. D. C. Walu. Aurangabad 431', 'unitprice': '4047', 'cnt': '.', 'price': '.'}}, {'nm': 'undergone Industrial Training', 'unitprice': '09/06/2022', 'cnt': 'to', 'price': '29/06/2022', 'sub': {'nm': 'Formulation unit at B-20, M.I.D.C., WaluJ Aurangabad.'}}, {'nm': 'Nuleh Natalat'}, {'nm': 'Human Resouces'}, {'nm': 'IndocoRemedies Ltd.'}, {'nm': 'B-20, Walu, Aurangabad'}], 'sub_total': {'subtotal_price': '29th June 2022'}, 'total': {'total_price': '99.0', 'creditcardprice': '190MH1947PLC005913'}}
